# NOTEBOOK FOURRE-TOUT pour scrips PlaMADE

In [190]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import os, re
import pandas as pd
import geopandas as gp
import numpy as np
import zipfile
import Connexion_Transfert as ct
import Outils as O
from geoalchemy2 import Geometry,WKTElement
from shapely.geometry import Point, LineString

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [159]:
from shapely import speedups
speedups.disable()

In [ ]:
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees'
#dézipper tous les dossier de Gérard
for root,dirs, files in os.walk(dossierSrc) : 
    for f in files : 
        if f.endswith('.zip') : 
            cheminFichier=os.path.join(root, f)
            print(cheminFichier[:-4])
            try:
                with zipfile.ZipFile(cheminFichier) as z:
                    z.extractall(cheminFichier[:-4])
                    print(f"fichier extrait {cheminFichier[:-4]}")
            except:
                print(f"pb extraction sur dossier {cheminFichier[:-4]}")

## 1. creer une bdd des fichiers geostandardises

In [ ]:
#pour traiter tous les fichiers d'un dossier général.suppose que la structure des tables a déjà été crée (par exemple avec l'import des des fichiers pouis truncate)
dossierSrc=r'D:\Boulot\PlaMADE\Ile-de-france\75-Paris\75'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
#dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne-Rhone-Alpes\Donnees_geostandardisees\Route01_v2_dec2020'
listEreur=[]
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
        for f in files :
            if f.endswith(('.shp', '.dbf')) : 
                print(os.path.join(root,f))
                if f.endswith('.shp') and 'N_ROUTIER_TRONCON' in f :
                    try :
                        ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID=None,geotype='MULTILINESTRINGZ', dims=3, creationMode='-append -update',encodageClient='UTF-8', version_simple=True)
                    except Exception as e: 
                        listEreur.append({'fichier': f, erreur : e})
                else : 
                    for fich,t in coupleFichierTable :
                        try : 
                            if fich in f.upper() and f.endswith('.dbf') : 
                                ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                                  schema='geostandardise_src', table=t, SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)
                            elif fich in f.upper() and f.endswith('.csv') :
                                df = pd.read_csv(os.path.join(root,f), 
                                                 keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                        except Exception as e: 
                            listEreur.append({'fichier': f, 'erreur' : e})

In [ ]:
#si les donnees trafic, vts, allure , rvt sont en csv, le plus simple c'est pandas : 
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
            for f in files :
                if f.endswith('.csv') :
                    for fich,t in coupleFichierTable :
                        if fich in f.upper() : 
                            try : 
                                print(os.path.join(root,f))
                                df = pd.read_csv(os.path.join(root,f), 
                                                keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                dfref=pd.read_sql(f'select * from geostandardise_src.{t} limit 1',c.sqlAlchemyConn)
                                colonnesEnTrop=[c.lower() for c in df.columns if c not in dfref.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                            except Exception as e :
                                print(f'Erreur sur : {os.path.join(root,f)} : {e} ')

In [ ]:
#pour test sur un dept
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne\35\N_ROUTIER_TRONCON_L_035.shp',
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID='2154',geotype='MULTILINESTRING', dims=3, creationMode='-append -update',encodageClient='UTF-8')

In [ ]:
#sur un fichier
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf',
                                                  schema='geostandardise_src', table='allure_national', SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)


## 2. Générer un fichier unique trafic RD
> L'idée est de se baser sur le fichier de résumé des fichiers gestionnaires et de leurs attributs pour produire un fichier concaténé, avec des attributs uniques relatifs au :
- tmja
- pcpl
- nom de la voie
- annee du trafic
- fichier source (sur Box internet)
- nom de la source

> ### 2.1 Principe mise en forme des fichiers
>> Pour faire ça on va aller chercher le [fichier de synthses des données RD] (https://cerema.app.box.com/folder/132749694470/Synthese_type_fichier.ods).
dans l'onglet trafic, on va itérer sur chaque ligne (donc récupération du tuple de valeur) : 
>> On limite l'analyse aux données présentant un colonne valide='oui' & type='sig' & type_geom='ligne'  
1. on récupere la valeur de la colonne "nom_fichier_trafic" :
    1. si l'extenstion est présente, on va lire le fichier (attention, via box Drive, donc paramètre de raw string en entrée)
    1. l'attribut relatif au tmja est toujours présent, celui au pc_pl parfois, comme pour les routes ou l'année, mais comme la fonction rename s'en fout, on fait le rename des 4 colonnes
    1. si des formules sont à appliquer, on les applique (liées au pcpl, et nom de route notamment
    1. gérer le format des dates pour l'année de mesure
    1. ajouter les attributs sur le fichiers sources, le nom de la source, le type de source
    1. enregistrer le fichier mise en forme dans le dossier qui va bien (paramètre entrée)

In [179]:
# ouvrir le fichiers de syntheses des données
dossierDonneesRd=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD'
fichierSynthese=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD\Synthese_type_fichier.ods'
dfFichierSynthese=pd.read_excel(fichierSynthese, sheet_name='trafic',engine='odf', nrows=35, dtype={'dept':str})

In [182]:
#creer le generateurde parcours des valeusr en tuple
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')
                                 & (dfFichierSynthese['regles_gestion']=='non')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')
                                 & (dfFichierSynthese['regles_gestion']!='non')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')].copy()

In [152]:
#fonction d'ajout des sources de données
def ajouterSourceDonnees(tupleNomAttr, dfFichierRenomme, fihcierSource) : 
    """
    ajouter les nomsource, fichier source et type de source aux donnees mise en formes
    in : 
        tupleNomAttr : tuple des noms d'attributs recherche dans le fchier source
        dfFichierRenomme : gdf : contient la trsucture attributaire prevue, apres calcul
        fihcierSource : rawString : chemin du fichier source
    """
    dfFichierRenomme['nomsource']='CD_'+tupleNomAttr.dept
    dfFichierRenomme['fichiersource']=fihcierSource

In [176]:
#fonction de calcul regroupant les traitements propres a chaque dept ayant des regles de gestion
def calculSpecifique(dfFichierSource, tupleNomAttr, listAttrNonNull) : 
    """
    calculer les valeur de tmja, pcpl, nom route et année dans les cas particuklier ou des regles de gestion sont necessaire
    in : 
        dept : string : departement sur 3 caracteres
        dfFichierSource : gdf : gdf issue des fichiers osurces gestionnaiere
        tupleNomAttr : tuple des noms d'attributs recherche dans le fchier source
        listAttrNonNull : list des attributs present dans le fichier (ayant une valuer non nulle)
    out : 
        dfFichierRenomme : gdf : contient la trsucture attributaire prevue, apres calcul
    """
    if tupleNomAttr.dept=='012' : 
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TMJATV2015'] if not pd.isnull(x['TMJATV2015']) else x['TMJA_TV'], axis=1)
        dfFichierSource['annee']=dfFichierSource.apply(lambda x : re.match('2[0-9]{3]',x['COMMENTAIR']) if not pd.isnull(x['COMMENTAIR']) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={tupleNomAttr.nom_attr_pcpl:'pcpl', tupleNomAttr.nom_attr_route:'nomRoute'})
    elif tupleNomAttr.dept=='037' :
        dfFichierSource['nomRoute']=dfFichierSource.idroute.apply(lambda x : x.split('_')[1])
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='049' :
        dfFichierSource['nomRoute']=dfFichierSource.Route.apply(lambda x : O.epurationNomRoute(x[3:]))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='052' :
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : x['MJA_PL']/x['MJA_TV']*100 if not pd.isnull(x['MJA_TV']) else None, axis=1)
        dfFichierSource['nomRoute']=dfFichierSource.Route.apply(lambda x : O.epurationNomRoute(x))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='058' :
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TRAFFIC_VL']+x['TRAFFIC_PL'], axis=1)
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : x['TRAFFIC_PL']/(x['TRAFFIC_VL']+x['TRAFFIC_PL'])*100 if x['TRAFFIC_VL']+x['TRAFFIC_PL']>0 else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={tupleNomAttr.nom_attr_route:'nomRoute', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='059' :
        dfFichierSource['nomRoute']=dfFichierSource.VOIE.apply(lambda x : O.epurationNomRoute(x[1:]))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='060' :
        dfFichierSource['pcpl']=dfFichierSource.PCT_PL_SEM.apply(lambda x : float(x.replace('%PL','')) if not x=='%PL')
        dfFichierSource['annee']=dfFichierSource.DATE_COMPT.apply(lambda x : pd.to_datetim(x).year)
    return dfFichierRenomme[listAttrNonNull]

In [ ]:
#Concat des donnees 
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            if f.lower() == t.nom_fichier_trafic.lower() : 
                cheminFichierDonnees=os.path.join(root, f)
                dfFichierSource=gp.read_file(cheminFichierDonnees)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('012', '037', '049', '052', '058', '059'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                elif t.dept in ('060', '075', '077', '082', '083') :
                    continue
                    #departements a traiter par la suite
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, cheminFichierDonnees)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatSansRegleGestion=pd.concat(dicoGdf.values())

In [218]:
#export à l'arrache pas le biais d'une bdd permissive car pb export avec Fiona
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    if dfConcatSansRegleGestion.geometry.name!='geom':
        df=O.gp_changer_nom_geom(dfConcatSansRegleGestion.loc[dfConcatSansRegleGestion.geometry.apply(lambda x : not pd.isnull(x))], 'geom')
        
        df.geom=df.apply(lambda x : WKTElement(x['geom'].wkt, srid=2154), axis=1)
        df.to_sql('concat_cds',c.sqlAlchemyConn,schema='donnees_sources',if_exists='append', index=False,
                  dtype={'geom': Geometry()} )

c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
